In [ ]:
import torchaudio
import torch
import librosa
import numpy as np
!pip install gTTS
from gtts import gTTS
from scipy.io import wavfile
from scipy.signal import stft
import nltk
from nltk.corpus import words

nltk.download('words')
nltk.download('punkt')





[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Function to preprocess text input
def preprocess_text(text):
    # Tokenize the text into words
    words_list = nltk.word_tokenize(text)

    # Filter out non-English words
    english_vocab = set(words.words())
    english_words = [word for word in words_list if word.lower() in english_vocab]

    # Join the English words back into text
    processed_text = ' '.join(english_words)

    return processed_text

In [ ]:
# Function to convert text to mel spectrogram
def text_to_melspectrogram(text, sample_rate=22050, n_fft=1024, hop_length=256, n_mels=80):
    # Convert text to speech and save to file
    tts = gTTS(text=text, lang='en')
    tts.save("temp.mp3")

    # Load the audio and compute mel spectrogram using librosa
    waveform, _ = librosa.load("temp.mp3", sr=sample_rate)
    mel_spec = librosa.feature.melspectrogram(y=waveform, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    mel_spec = librosa.power_to_db(mel_spec, ref=np.max)

    return torch.tensor(mel_spec)



In [ ]:
# Function to convert mel spectrogram to audio waveform
def melspectrogram_to_audio(mel_spec, sample_rate=22050, n_fft=1024, hop_length=256):
    # Invert mel spectrogram back to waveform using librosa
    mel_spec = librosa.db_to_power(mel_spec.numpy())
    waveform = librosa.feature.inverse.mel_to_audio(mel_spec, sr=sample_rate, n_fft=n_fft, hop_length=hop_length)

    return torch.tensor(waveform)
# Function to save audio waveform to file
def save_audio(waveform, output_path, sample_rate=22050):
    wavfile.write(output_path, sample_rate, waveform.numpy())



In [ ]:

# Receive user input
user_text = input("Enter the text you want to convert to speech: ")
user_text = preprocess_text(user_text)

# Convert text to mel spectrogram
mel_spec = text_to_melspectrogram(user_text)

# Convert mel spectrogram to audio waveform
audio_waveform = melspectrogram_to_audio(mel_spec)

# Save audio waveform to file
output_path = "output_audio.wav"
save_audio(audio_waveform, output_path)

# Download the synthesized audio
from google.colab import files
files.download(output_path)

Enter the text you want to convert to speech: hello


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>